In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "knn7d"
fs_method, fs_ratio = "fcls", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9141 | AUPR: 0.6546 | Acc: 95.39%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9180 | AUPR: 0.6638 | Acc: 95.39%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9206 | AUPR: 0.6712 | Acc: 95.43%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9134 | AUPR: 0.6510 | Acc: 95.33%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9173 | AUPR: 0.6619 | Acc: 95.39%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9201 | AUPR: 0.6723 | Acc: 95.47%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9206 | AUPR: 0.6712 | Acc: 95.43%
Internal test: AUC: 0.9297 | AUPR: 0.6979 | Acc: 95.28%
External test: AUC: 0.9206 | AUPR: 0.6665 | Acc: 95.16%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9136 | AUPR: 0.6520 | Acc: 95.40%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9176 | AUPR: 0.6621 | Acc: 95.43%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9203 | AUPR: 0.6716 | Acc: 95.52%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9135 | AUPR: 0.6538 | Acc: 95.33%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9169 | AUPR: 0.6629 | Acc: 95.39%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9199 | AUPR: 0.6707 | Acc: 95.40%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9203 | AUPR: 0.6716 | Acc: 95.52%
Internal test: AUC: 0.9279 | AUPR: 0.6936 | Acc: 95.18%
External test: AUC: 0.9208 | AUPR: 0.6667 | Acc: 95.17%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9149 | AUPR: 0.6480 | Acc: 95.31%
model(C=10.0)
Valid:         AUC: 0.9147 | AUPR: 0.6480 | Acc: 95.31%
model(C=100.0)
Valid:         AUC: 0.9146 | AUPR: 0.6479 | Acc: 95.30%
best model(C=1.0)
Valid:         AUC: 0.9149 | AUPR: 0.6480 | Acc: 95.31%
Internal test: AUC: 0.9225 | AUPR: 0.6820 | Acc: 95.16%
External test: AUC: 0.9129 | AUPR: 0.6313 | Acc: 94.85%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9087 | AUPR: 0.6346 | Acc: 95.27%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9091 | AUPR: 0.6413 | Acc: 95.29%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.8950 | AUPR: 0.6153 | Acc: 95.24%
best model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9091 | AUPR: 0.6413 | Acc: 95.29%
Internal test: AUC: 0.9137 | AUPR: 0.6651 | Acc: 95.09%
External test: AUC: 0.9016 | AUPR: 0.5961 | Acc: 94.83%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9106 | AUPR: 0.6539 | Acc: 89.12%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9115 | AUPR: 0.6544 | Acc: 89.24%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9131 | AUPR: 0.6563 | Acc: 89.45%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9114 | AUPR: 0.6568 | Acc: 89.54%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9119 | AUPR: 0.6561 | Acc: 89.40%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9130 | AUPR: 0.6573 | Acc: 89.52%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9114 | AUPR: 0.6568 | Acc: 89.54%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9119 | AUPR: 0.6561 | Acc: 89.40%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9130 | AUPR: 0.6573 | Acc: 89.52%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9131 | AUPR: 0.6563 | Acc: 8

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9142 | AUPR: 0.6251 | Acc: 95.08%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9134 | AUPR: 0.6175 | Acc: 95.04%
model(kernel=linear, C=1)
Valid:         AUC: 0.9127 | AUPR: 0.6140 | Acc: 94.96%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9142 | AUPR: 0.6251 | Acc: 95.08%
Internal test: AUC: 0.9233 | AUPR: 0.6640 | Acc: 94.96%
External test: AUC: 0.9156 | AUPR: 0.6101 | Acc: 94.57%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9183 | AUPR: 0.6532 | Acc: 95.25%
model(criterion=squared_error)
Valid:         AUC: 0.9183 | AUPR: 0.6515 | Acc: 95.27%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9183 | AUPR: 0.6532 | Acc: 95.25%
Internal test: AUC: 0.9275 | AUPR: 0.6932 | Acc: 95.30%
External test: AUC: 0.9201 | AUPR: 0.6507 | Acc: 95.15%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9112 | AUPR: 0.6219 | Acc: 95.07%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9126 | AUPR: 0.6248 | Acc: 95.06%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9129 | AUPR: 0.6254 | Acc: 95.06%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9129 | AUPR: 0.6254 | Acc: 95.06%
Internal test: AUC: 0.9233 | AUPR: 0.6882 | Acc: 95.28%
External test: AUC: 0.9132 | AUPR: 0.6214 | Acc: 94.62%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7667 | AUPR: 0.4438 | Acc: 94.87%
model(n_neighbors=5)
Valid:         AUC: 0.7904 | AUPR: 0.4882 | Acc: 95.14%
model(n_neighbors=7)
Valid:         AUC: 0.8057 | AUPR: 0.5140 | Acc: 95.22%
best model(n_neighbors=7)
Valid:         AUC: 0.8057 | AUPR: 0.5140 | Acc: 95.22%
Internal test: AUC: 0.8095 | AUPR: 0.5217 | Acc: 95.18%
External test: AUC: 0.8158 | AUPR: 0.5234 | Acc: 95.05%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8666 | AUPR: 0.5597 | Acc: 87.66%
model(criterion=log_loss)
Valid:         AUC: 0.8708 | AUPR: 0.5726 | Acc: 87.53%
model(criterion=entropy)
Valid:         AUC: 0.8708 | AUPR: 0.5726 | Acc: 87.53%
best model(criterion=log_loss)
Valid:         AUC: 0.8708 | AUPR: 0.5726 | Acc: 87.53%
Internal test: AUC: 0.8701 | AUPR: 0.6138 | Acc: 86.08%
External test: AUC: 0.8597 | AUPR: 0.5790 | Acc: 86.06%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8748 | AUPR: 0.4235 | Acc: 86.84%
best model()
Valid:         AUC: 0.8748 | AUPR: 0.4235 | Acc: 86.84%
Internal test: AUC: 0.8850 | AUPR: 0.4486 | Acc: 87.03%
External test: AUC: 0.8707 | AUPR: 0.4031 | Acc: 85.50%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)